## 0208

In [ ]:
import pandas as pd
import re

data = "dbpedia_14"
text_len = 60
model = "transformeral"
layer = 5
epoch = 10
lr = 0.0001
fix_previous_layer = True
#for mask in range(1,1+layer):
mask = 5
save_path = f"ckpt/{data}_{model}_l{layer}_pad{text_len}_m{mask}/"  
#load_path = f"ckpt/{data}_{model}_l{layer}_pad{text_len}_m{mask-1}/" 
out_path = f"result/0117/fix train adapt/{data}_{model}_l{layer}ad_pad{text_len}_m{mask}/"
log = f"{out_path}/{data}_{model}_l{layer}.log"

with open(log,mode='r') as log:
    buffer = log.readlines()
    df = pd.DataFrame(buffer,columns=["log"])
    df

In [ ]:
class history(object):
    def __init__(self):
        self.auc = {}
        self.acc = {}
        self.entr = {}
        for threshold in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
            self.auc[threshold] = []
            self.acc[threshold] = []
            self.entr[threshold] = []
            
result = history()
for line in buffer:
    match = re.match('Test Epoch(.)*', line)
    if match!=None:
        print(match.group())
        match = re.findall("\d?\.\d+",match.group(0))
        print(match)
        threshold = float(match[0])
        result.acc[threshold].append(match[1])
        result.auc[threshold].append(match[2])
        result.entr[threshold].append(match[3])

In [ ]:
for threshold in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
    print(len(result.acc[threshold]))
    print(len(result.auc[threshold]))
    print(len(result.entr[threshold]))

In [ ]:
### 2/06 - valid/test 80%:20%
###      - prefix
###      - plot result

import pandas as pd
import re

data = "ag_news"
text_len = 175


for model in ["linearal","lstmal","transformeral"]:
    result = [[] for _ in range(4)]
    for threshold in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:

        layer = 5
        epoch = 10
        lr = 0.0001
        fix_previous_layer = False
        
        mask = 5
        out_path = f"result/0208/test/prefix/{data}_{model}_l{layer}adp_pad{text_len}_t{threshold}_m{mask}_test/"
        log = f"{out_path}/{data}_{model}_l{layer}.log"
            
        with open(log,mode='r') as log:
            buffer = log.readlines()
            df = pd.DataFrame(buffer,columns=["log"])
            df
            
            for line in buffer:
                match = re.match('Test threshold(.)*', line)
                if match!=None:
                    match = re.findall("\d?\.\d+",match.group(0))
                    #print(match[1])
                    result[0].append(float(match[1]))
                    result[1].append(float(match[2]))
                    result[2].append(float(match[3]))
            for line in buffer:
                match = re.match('t(.)*_test_time(.)*', line)
                if match!=None:
                    match = re.findall("\d?\.\d+",match.group(0))
                    #print(match[1])
                    result[3].append(float(match[1]))
    print("\n",model)
    for i in range(4):
        print(i)
        for v in result[i]:
            print(v)                    

                    

In [ ]:
### 2/06 - valid/test 80%:20%
###      - fix
###      - plot result

import pandas as pd
import re

data = "ag_news"
text_len = 175

for model in ["linearal","lstmal","transformeral"]:
    result = [[] for _ in range(4)]
    for threshold in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:

        layer = 5
        epoch = 10
        lr = 0.0001
        fix_previous_layer = False
        
        mask = 5
        out_path = f"result/0208/test/fix/{data}_{model}_l{layer}adf_pad{text_len}_t{threshold}_m{mask}_test/"
        log = f"{out_path}/{data}_{model}_l{layer}.log"
            
        with open(log,mode='r') as log:
            buffer = log.readlines()
            df = pd.DataFrame(buffer,columns=["log"])
            df
            
            for line in buffer:
                match = re.match('Test threshold(.)*', line)
                if match!=None:
                    match = re.findall("\d?\.\d+",match.group(0))
                    #print(match[1])
                    result[0].append(float(match[1]))
                    result[1].append(float(match[2]))
                    result[2].append(float(match[3]))
            for line in buffer:
                match = re.match('t(.)*_test_time(.)*', line)
                if match!=None:
                    match = re.findall("\d?\.\d+",match.group(0))
                    #print(match[1])
                    result[3].append(float(match[1]))
    print("\n",model)
    for i in range(4):
        print(i)
        for v in result[i]:
            print(v)                    

                    

## 0214

In [ ]:
### 0214
### pad normal_al adapt 
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
total_result = pd.DataFrame(columns=['padding_size', 'model', 'test_threshold', 'test_acc', 'test_auc',
       'test_entr', 'test_time'])


data = "ag_news"
#for model in ["lstmal"]:
for model in ["lstmal","linearal","transformeral"]:
    #for text_len in [25,50,75,100,125,150,175]:
    
    for text_len in [25,50,75,100,125,150,175]:
        layer = 5
        epoch = 50
        lr = 0.0001
        save_path = f"ckpt/{data}_{model}_l{layer}ad_pad{text_len}/"
        out_path = f"result/0214/al adapt/{data}_{model}_l{layer}ad_pad{text_len}/"
        log = f"{out_path}/{data}_{model}_l{layer}.log"
        
        result = pd.DataFrame(columns=["padding_size"])
        result_ep = pd.DataFrame(columns=["epoch","ep_train_time"])
        result_ep["epoch"] = [*range(50)]
        list_acc = []
        list_auc = []
        list_entr = []
        list_test_time = []
        list_ep_train_time = []
        list_ep_train_acc = []
        list_ep_train_auc = []
        list_ep_train_entr = []
        list_ep_valid_time = []
        list_ep_valid_acc = []
        list_ep_valid_auc = []
        list_ep_valid_entr = []
        train_time = 0
        init_time = 0
        with open(log,mode='r') as log:
            buffer = log.readlines()
            df = pd.DataFrame(buffer,columns=["log"])
            df
            
            
            for line in buffer:
                match = re.match('Test threshold(.)*', line)
                if match!=None:
                    match = re.findall("\d?\.\d+",match.group(0))
                    list_acc.append(float(match[1]))
                    list_auc.append(float(match[2]))
                    list_entr.append(float(match[3]))

                match = re.match('t(.)*_test_time(.)*', line)
                if match!=None:
                    match = re.findall("\d+\.\d+",match.group(0))
                    list_test_time.append(float(match[1]))

                match = re.match('init_time(.)*', line)
                if match!=None:
                    match = re.findall("\d+\.\d+",match.group(0))
                    init_time = float(match[0])
                match = re.match('(.)*valid_time(.)*', line)
                if match!=None:
                    match = re.findall("\d+\.\d+",match.group(0))
                    train_time = float(match[0])
                    
                match = re.match('ep(\d)+_train_time(.)*', line)
                if match!=None:
                    match = re.findall("\d+\.\d+",match.group(0))
                    list_ep_train_time.append(float(match[0]))
                    
                match = re.match('ep(\d)+_t(.)*_test_time(.)*', line)
                if match!=None:
                    match = re.findall("\d+\.\d+",match.group(0))
                    list_ep_valid_time.append(float(match[1]))
                    
                match = re.match('Train Epoch\d+ Acc(.)*', line)
                if match!=None:
                    match = re.findall("\d+\.\d+",match.group(0))
                    list_ep_train_acc.append(match[0])
                    list_ep_train_auc.append(match[1])
                    
                match = re.match('Test Epoch\d+ threshold(.)*', line)
                if match!=None:
                    match = re.findall("\d+\.\d+",match.group(0))
                    #print(line)
                    #print(match)
                    list_ep_valid_acc.append(match[1])
                    list_ep_valid_auc.append(match[2])
                    list_ep_valid_entr.append(match[3])
                    
        ep_valid_time = np.array(list_ep_valid_time).reshape((50,-1))
        ep_valid_acc = np.array(list_ep_valid_acc).reshape((50,-1))
        ep_valid_auc = np.array(list_ep_valid_auc).reshape((50,-1))
        ep_valid_entr = np.array(list_ep_valid_entr).reshape((50,-1))
        
        result_ep["ep_train_acc"] = list_ep_train_acc 
        result_ep["ep_train_auc"] = list_ep_train_auc          
        
        result_ep["ep_valid_time"] = np.sum(ep_valid_time,-1) 
        result_ep[[f"ep_valid.{(i+1)}_time" for i in range(9)]] = ep_valid_time
        result_ep["ep_train_time"] = list_ep_train_time 
        
        
        result['model'] = [model]*9
        result['padding_size'] = [text_len]*9
        result["test_threshold"] = [(1+i)*0.1 for i in range(9)]
        result["test_acc"] = list_acc
        result["test_auc"] = list_auc
        result["test_avg_entr"] = list_entr
        result["test_time"] = list_test_time
        result["init_time"] = [init_time]*9
        result["train+valid_time"] = [train_time]*9
        result["train_time"] = [result_ep["ep_train_time"].sum()]*9
        result["valid_time"] = [result_ep["ep_valid_time"].sum()]*9
        #print(result)
        print(result_ep)
    
        title = f"{data}_{model}_l{layer}ad_pad{text_len}"
        t = [f"0.{i+1}" for i in range(9)]
        
        plt.title(title)
        plt.plot(t,result["test_acc"],"o-") 
        plt.xlabel("threshold")
        plt.ylabel("test acc")
        plt.savefig(f"result/_csv/{title}_plt_acc.png", bbox_inches = "tight",facecolor='white', transparent=False)
        plt.show()

        plt.title(title)
        plt.plot(t,result["test_auc"],"o-") 
        plt.xlabel("threshold")
        plt.ylabel("test auc")
        plt.savefig(f"result/_csv/{title}_plt_auc.png", bbox_inches = "tight",facecolor='white', transparent=False)
        plt.show()
        
        plt.title(title)
        plt.plot(t,result["test_avg_entr"],"o-") 
        plt.xlabel("threshold")
        plt.ylabel("test avg entr")
        plt.savefig(f"result/_csv/{title}_plt_entr.png", bbox_inches = "tight",facecolor='white', transparent=False)
        plt.show()
        
        result.to_csv(f"result/_csv/{title}.csv", index=False)
        result_ep.to_csv(f"result/_csv/{title}_ep.csv", index=False)
        total_result = pd.concat([total_result,result],axis=0,ignore_index=True)            
        del result

In [ ]:
total_result[total_result["model"]=="lstmal"]
###
# TODO: plot acc/thres, auc/thres
###

In [ ]:
data = "ag_news"
for model in ["lstmal","linearal","transformeral"]:
    for threshold in [.1,.2,.3,.4,.5,.6,.7,.8,.9]:
        for text_len in [175]:
            layer = 5
            epoch = 10
            lr = 0.0001
            
            result = pd.DataFrame(columns=["padding_size"])
            result_ep = pd.DataFrame(columns=["epoch","ep_train_time"])
            result_ep["epoch"] = [*range(50)]
            list_acc = []
            list_auc = []
            list_entr = []
            list_test_time = []
            list_init_time = []
            list_train_time = []
            
            for mask in range(1,1+layer):
                save_path = f"ckpt/{data}_{model}_l{layer}adf_pad{text_len}_t{threshold}_m{mask}/"  
                load_path = f"ckpt/{data}_{model}_l{layer}adf_pad{text_len}_t{threshold}_m{mask-1}/" 
                out_path = f"result/{data}_{model}_l{layer}adf_pad{text_len}_t{threshold}_m{mask}/"
                log = f"{out_path}/{data}_{model}_l{layer}.log"
                
                with open(log,mode='r') as log:
                    buffer = log.readlines()
                    df = pd.DataFrame(buffer,columns=["log"])
                    df
                    
                    
                    for line in buffer:
                        match = re.match('Test threshold(.)*', line)
                        if match!=None:
                            match = re.findall("\d?\.\d+",match.group(0))
                            list_acc.append(float(match[1]))
                            list_auc.append(float(match[2]))
                            list_entr.append(float(match[3]))

                        match = re.match('t(.)*_test_time(.)*', line)
                        if match!=None:
                            match = re.findall("\d+\.\d+",match.group(0))
                            list_test_time.append(float(match[1]))
                        match = re.match('init_time(.)*', line)
                        if match!=None:
                            match = re.findall("\d+\.\d+",match.group(0))
                            list_init_time.append(float(match[0]))
                        match = re.match('(.)*valid_time(.)*', line)
                        if match!=None:
                            match = re.findall("\d+\.\d+",match.group(0))
                            list_train_time.append(float(match[0]))
                                
            result['model'] = [model]*5
            result['padding_size'] = [text_len]*5
            result["train_mask"] = [(1+i) for i in range(5)]
            
            result["test_acc"] = list_acc
            result["test_auc"] = list_auc
            result["test_avg_entr"] = list_entr
            
            result["init_time"] = list_init_time
            result["test_time"] = list_test_time
            result["train+valid_time"] = list_train_time
        
            print(result)
            ass